In [ ]:
from pynq import Overlay, allocate, VideoMode
import cv2
import numpy as np
import os


overlay = Overlay("/home/xilinx/pynq/overlays/binarizer/binarizer.bit")
dma = overlay.axi_dma_0

vdma = overlay.axi_vdma_0
vdma.readchannel.mode = VideoMode(1280, 720, 32)
vdma.readchannel.start()

In [2]:
img = cv2.imread('/home/xilinx/images/input.jpg')
print('image loaded', img.shape)

image loaded (630, 1200, 3)


In [3]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
h, w = img_gray.shape

In [4]:
flat_img = img_gray.flatten().astype(np.uint8)

In [6]:
pad = 0
if len(flat_img) % 4 != 0:
    pad = 4 - (len(flat_img) % 4)
    flat_img = np.pad(flat_img, (0, pad), constant_values=0)
    print("Padded by:", pad, "pixels")

n_pixels = len(flat_img)
print('transfer size', n_pixels)

transfer size 756000


In [8]:
input_buffer = allocate(shape=(n_pixels,), dtype=np.uint8)
output_buffer = allocate(shape=(n_pixels,), dtype=np.uint8)

input_buffer[:] = flat_img

In [9]:
dma.recvchannel.transfer(output_buffer)
dma.sendchannel.transfer(input_buffer)

dma.recvchannel.wait()
dma.sendchannel.wait()

In [ ]:
if pad > 0:
    out_arr = np.array(output_buffer[:-pad])
else:
    out_arr = np.array(output_buffer)

out_img = out_arr.reshape((h, w))

cv2.imwrite('/home/xilinx/images/output.jpg', out_img)

out_img_rgba = cv2.cvtColor(out_img, cv2.COLOR_GRAY2RGBA)

True

In [ ]:
display_buffer = allocate(shape=(h, w, 4), dtype=np.uint8)
display_buffer[:] = out_img_rgba
display_buffer.flush()

vdma.readchannel.transfer(display_buffer)
vdma.readchannel.park(0)

In [ ]:
vdma.reachchannel.stop()
del input_buffer, output_buffer, display_buffer